# 1.0. Import

In [1]:
#Bibliotecas utilizadas no trabalho

import pandas as pd
import xgboost as xgb

from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel,RationalQuadratic
from sklearn.model_selection import KFold,cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline


from tensorflow import keras
from tensorflow.keras import layers,callbacks
import tensorflow as tf


In [2]:
#Importando o DataSet
df_concreto = pd.read_csv("./databases/train.csv")

In [3]:
df_concreto.head()

,Cimento,EscóriadeAltoForno,CinzasVolantes,Agua,Superplastificante,AgregadoGrosso,Agregadofino,Resistência
0,380.0,95.0,0.0,228.0,0.0,932.0,594.0,36.45
1,266.0,114.0,0.0,228.0,0.0,932.0,670.0,45.85
2,475.0,0.0,0.0,228.0,0.0,932.0,594.0,39.29
3,198.6,132.4,0.0,192.0,0.0,978.4,825.5,28.02
4,304.0,76.0,0.0,228.0,0.0,932.0,670.0,47.81


# 2.0. Machine Learning

## 2.1. Preparando para testar os Modelos

Primeiro vamos criar um DataFrame para armazenar o resultado de cada modelo.

In [4]:
colunas = ['MODEL', 'R2','RMSE','MAE','MAE_MAX','MAE_MIN']
Metrics = pd.DataFrame(columns = colunas)

Vamos também separar os dados de entrada e saída:

In [5]:
X = df_concreto.drop(['Resistência'], axis=1)
y = df_concreto['Resistência']

Definindo as configurações do KFold:

In [6]:
kf = KFold(10, shuffle=True, random_state=451)

Criando uma função para calcular as metricas:

In [7]:
def calcular_metricas(y_true, y_pred, name_model):
    MAE = mean_absolute_error(y_true, y_pred)              
    RMSE = mean_squared_error(y_true, y_pred, squared=False)    
    R2 = r2_score(y_true, y_pred) 
    MAE_MAX = abs(y_pred-y_true).max()
    MAE_MIN = abs(y_pred-y_true).min()

    return {'MODEL': name_model,'MAE': MAE,'RMSE': RMSE, 'R2': R2,
                        'MAE_MAX': MAE_MAX,'MAE_MIN': MAE_MIN}  

## 2.2. Modelos
Obs: Antes dos treinos foi feito um fine-tuning dos modelos que está no notebook "Fine_tunning".

### 2.2.1. Decision Tree

In [8]:
xgbc = xgb.XGBRegressor(colsample_bytree= 0.898275838301962, 
                            learning_rate= 0.10335797738188568,
                            max_depth = 10,
                            min_child_weight= 3.5585146125243106, 
                            subsample= 0.7384101157071529)

pipeline = Pipeline([('transformer', StandardScaler()), ('estimator', xgbc)])

predict_xgb = cross_val_predict(pipeline, X, y, cv = kf)

Metrics = Metrics.append(calcular_metricas(y,predict_xgb,'XGBoost'), ignore_index=True)

### 2.2.2. SVR

In [9]:
svr_model = SVR(kernel='rbf', C=32.919679770421595, gamma = 0.6017663761743526)

pipeline = Pipeline([('transformer', StandardScaler()), ('estimator', svr_model)])

predict_svr = cross_val_predict(pipeline, X, y, cv = kf)

Metrics = Metrics.append(calcular_metricas(y,predict_svr,'SVR'), ignore_index=True)

### 2.2.3. RNA

In [10]:
df_analise_rna = pd.DataFrame(columns=[['real','predito']])

# Vamos criar um for para percorrer cada fold definida 
for linhas_treino, linhas_valid in kf.split(X):
    
    #Função para obter resultados reprodutiveis 
    tf.random.set_seed(1)

    X_treino, X_valid = X.iloc[linhas_treino], X.iloc[linhas_valid]
    y_treino, y_valid = y.iloc[linhas_treino], y.iloc[linhas_valid]
    
    scaler = StandardScaler()
    X_treino = scaler.fit_transform(X_treino)
    X_valid = scaler.transform(X_valid)
    
    # Vamos definir EarlyStopping. Isso faz com que o modelo pare quando parar de melhorar as predições
    early_stopping = callbacks.EarlyStopping(
        min_delta=0.05, # o valor de melhora que nos desejamos
        patience=15, # a quantidade de vezes que ele deve rodar sem obter a melhora acima para parar o algoritmo
        restore_best_weights=True)
    
    #Definimos o Modelo
    model = keras.Sequential([
        layers.Dense(512, activation='relu', input_shape=[X.shape[1]]),#Define a camada com 256 neuronios, função de ativação relu, input é quantas variaveis de entrada possui
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(1), #Saida de apenas uma variavel
    ])
    
    
    model.compile(
        optimizer='sgd', # define o tipo de otimizador utilizado
        loss='mae', # definimos a loss function
        metrics=['mae','mse'],
    )
    history = model.fit(
        X_treino, y_treino, #dados de treino
        validation_split=0.1, #dados de teste
        batch_size=45, #Define a quantidade de lotes
        epochs=150, #Define a quantidade de epocas
        verbose=0,
        callbacks=[early_stopping],
    
    )
    
    rna_pred = model.predict(X_valid)
    df_pred = pd.DataFrame()    
    df_pred['real'] = y_valid
    df_pred['predito'] = rna_pred    
    
    if df_analise_rna.shape[0] == 0:
        df_analise_rna = df_pred
    else:
        df_analise_rna = pd.concat([df_analise_rna, df_pred])
    

# Vamos salvar as metricas no data frame "Metrics".
Metrics=Metrics.append(calcular_metricas(df_analise_rna['real'],df_analise_rna['predito'],'RNA'), ignore_index=True)

### 2.2.4. Gaussian

In [11]:
kernel = ConstantKernel() * RationalQuadratic()
gpr = GaussianProcessRegressor(kernel=kernel, alpha=0.23604724748727526,random_state=0)

pipeline = Pipeline([('transformer', StandardScaler()), ('estimator', gpr)])

predict_gpr = cross_val_predict(pipeline, X, y, cv = kf)

Metrics = Metrics.append(calcular_metricas(y,predict_gpr,'GPR'), ignore_index=True)

## 2.3. Resultado

In [12]:
Metrics

,MODEL,R2,RMSE,MAE,MAE_MAX,MAE_MIN
0,XGBoost,0.835290,3.454252,2.236298,18.776949,0.000022
1,SVR,0.798502,3.820574,2.262172,19.899261,0.000106
2,RNA,0.743760,4.308412,2.904185,28.003023,0.000240
3,GPR,0.831241,3.496443,1.959406,21.120264,0.000655


## 2.4. Piores Resultados

Criando uma tabela com todos os 10 piores resultados de cada modelo:

In [13]:
# Piores XGBoost
df_an_xgboost = df_concreto.copy()

df_an_xgboost['Predito'] = predict_xgb
df_an_xgboost['Erro_absoluto'] = abs(df_an_xgboost['Resistência']-df_an_xgboost['Predito'])

piores_xgb = df_an_xgboost.sort_values(by='Erro_absoluto', ascending=False).head(10)
piores_xgb['Model'] = 'XGBoost'

piores_xgb = piores_xgb.reindex(columns=['Model','Cimento', 'EscóriadeAltoForno', 'CinzasVolantes', 'Agua',
       'Superplastificante', 'AgregadoGrosso', 'Agregadofino', 'Resistência',
       'Predito', 'Erro_absoluto'])

# Piores SVR
df_an_svr = df_concreto.copy()

df_an_svr['Predito'] = predict_svr
df_an_svr['Erro_absoluto'] = abs(df_an_svr['Resistência']-df_an_svr['Predito'])
piores_svr = df_an_svr.sort_values(by='Erro_absoluto', ascending=False).head(10)

piores_svr['Model'] = 'SVR'

piores_svr = piores_svr.reindex(columns=['Model','Cimento', 'EscóriadeAltoForno', 'CinzasVolantes', 'Agua',
       'Superplastificante', 'AgregadoGrosso', 'Agregadofino', 'Resistência',
       'Predito', 'Erro_absoluto'])

# Piores RNA
df_an_rna = df_concreto.copy()

df_an_rna['Predito'] = df_analise_rna['predito']
df_an_rna['Erro_absoluto'] = abs(df_an_rna['Resistência']-df_an_rna['Predito'])

piores_rna = df_an_rna.sort_values(by='Erro_absoluto', ascending=False).head(10)

piores_rna['Model'] = 'RNA'

piores_rna = piores_rna.reindex(columns=['Model','Cimento', 'EscóriadeAltoForno', 'CinzasVolantes', 'Agua',
       'Superplastificante', 'AgregadoGrosso', 'Agregadofino', 'Resistência',
       'Predito', 'Erro_absoluto'])

# Piores GPR
df_an_gaus = df_concreto.copy()

df_an_gaus['Predito'] = predict_gpr
df_an_gaus['Erro_absoluto'] = abs(df_an_gaus['Resistência']-df_an_gaus['Predito'])

piores_gp = df_an_gaus.sort_values(by='Erro_absoluto', ascending=False).head(10)

piores_gp['Model'] = 'Gaussian'

piores_gp = piores_gp.reindex(columns=['Model','Cimento', 'EscóriadeAltoForno', 'CinzasVolantes', 'Agua',
       'Superplastificante', 'AgregadoGrosso', 'Agregadofino', 'Resistência',
       'Predito', 'Erro_absoluto'])

In [14]:
dframes = [piores_xgb, piores_svr, piores_gp,piores_rna]

piores = pd.concat(dframes)

In [15]:
piores

,Model,Cimento,EscóriadeAltoForno,CinzasVolantes,Agua,Superplastificante,AgregadoGrosso,Agregadofino,Resistência,Predito,Erro_absoluto
49,XGBoost,200.0,200.0,0.0,190.0,0.0,1145.0,660.0,49.25,30.473051,18.776949
73,XGBoost,436.0,0.0,0.0,218.0,0.0,838.4,719.7,23.85,38.463768,14.613768
35,XGBoost,277.1,0.0,97.4,160.6,11.8,973.9,875.6,48.28,35.186733,13.093267
176,XGBoost,165.0,0.0,150.0,182.0,12.0,1023.0,729.0,18.03,29.091600,11.061600
65,XGBoost,194.7,0.0,100.5,170.2,7.5,998.0,901.8,37.27,26.873520,10.396480
27,XGBoost,212.0,0.0,124.8,159.0,7.8,1085.4,799.5,38.50,29.179279,9.320721
55,XGBoost,165.0,128.5,132.1,175.1,8.1,1005.8,746.6,46.39,37.357944,9.032056
38,XGBoost,249.1,0.0,98.8,158.1,12.8,987.8,889.0,30.85,39.847107,8.997107
39,XGBoost,297.2,0.0,117.5,174.8,9.5,1022.8,753.5,47.40,38.519932,8.880068
307,XGBoost,164.6,0.0,150.4,181.6,11.7,1023.3,728.9,18.03,26.747835,8.717835


Podemos perceber que a linha 49 é a pior para todos os modelos e a 73 aparece entre as 10 piores para todos os modelos.

## 2.5. Feature Importance

Analisando a feature importance disponivel para o modelo XGBoost.

In [16]:
xgbc = xgb.XGBRegressor(colsample_bytree= 0.898275838301962, 
                            learning_rate= 0.10335797738188568,
                            max_depth = 10,
                            min_child_weight= 3.5585146125243106, 
                            subsample= 0.7384101157071529)
xgbc.fit(X, y)
     
cim = xgbc.feature_importances_[0]
esc = xgbc.feature_importances_[1]
cin = xgbc.feature_importances_[2]
agu = xgbc.feature_importances_[3]  
sup = xgbc.feature_importances_[4]   
gra = xgbc.feature_importances_[5]    
miu = xgbc.feature_importances_[6]

In [17]:
importances = pd.Series(data=(cim,esc,cin,agu,sup,gra,miu), index=X.columns).sort_values(ascending=False)

In [18]:
importances

Cimento               0.236191
EscóriadeAltoForno    0.169150
CinzasVolantes        0.148831
Agua                  0.133787
Superplastificante    0.120237
AgregadoGrosso        0.104097
Agregadofino          0.087706
dtype: float32